In [2]:
import paramiko 
import datetime
import pandas as pd
import os
import datetime
import glob
print(datetime.datetime.now())
print(os.getcwd())

2020-09-17 10:10:54.715377
/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploading_Scripts


In [4]:
list_pyd_removed_quarterly_control=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/*.csv")
list_pyd_removed_quarterly_control=[x for x in list_pyd_removed_quarterly_control if "_A.csv" in x]
list_pyd_removed_quarterly_control

['/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Active_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Lapsed_A.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_clients/removed_Q3_QuarterlyControl_09142020_PYD_Signup_NoPurchase_A.csv']

In [14]:
for file in list_pyd_removed_quarterly_control:
    segment_name=os.path.basename(file).split("9142020_")[1][:-4]
    df=pd.read_csv(file,dtype=str)
    df=df[['customersummary_emailhash','customersummary_c_primaryscnhash']].drop_duplicates()
    df=df[['customersummary_c_primaryscnhash','customersummary_emailhash']]
    df=df.rename(columns={'customersummary_c_primaryscnhash':'customer_id_hashed',"customersummary_emailhash":"email_address_hash"})
    df['segment_2020_Sept_PYD']=segment_name
    df.to_csv("/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_LR_format/%s.csv"%segment_name,index=False)
    print(segment_name,datetime.datetime.now(),df.shape)

PYD_Active_A 2020-09-17 10:30:09.992865 (6473944, 3)
PYD_Lapsed_A 2020-09-17 10:30:21.919249 (1174859, 3)
PYD_Signup_NoPurchase_A 2020-09-17 10:30:37.746960 (971806, 3)


In [15]:
list_files_LR_to_Upload=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2020_Q3/Audience_for_21mm_Act_Goal/Uploaded_Files/PYD_LR_format/*.csv")
pd.read_csv(list_files_LR_to_Upload[0],dtype=str,nrows=2)

,customer_id_hashed,email_address_hash,segment_2020_Sept_PYD
0,26e9f762bad8fef8236cfc8cc2295438688f96a680d21c...,a18b86772997a332cfdb345d3b040a314a6631e1ea73ae...,PYD_Active_A
1,a41d61718ae691378e10e77918e29d5761ed7a8361aad7...,ec8fe455ffeed633d1c5e9578f7bdda11a9d8a0ad88be8...,PYD_Active_A


In [16]:
# Uploading

import paramiko
host = "files.liveramp.com"
port = 22
password = "Biglots2018!"
username = "lr-big-lots"

transport = paramiko.Transport((host, port))

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [17]:
for local_file in list_files_LR_to_Upload:
    base_name=os.path.basename(local_file)
    remote_name="/uploads/big_lots_onboarding/"+base_name
    sftp.put(local_file,remote_name)
    print(datetime.datetime.now(),remote_name)
print("All_Done: "+str(datetime.datetime.now()))
sftp.close()
transport.close()

2020-09-17 10:33:04.887418 /uploads/big_lots_onboarding/PYD_Active_A.csv
2020-09-17 10:33:29.835051 /uploads/big_lots_onboarding/PYD_Lapsed_A.csv
2020-09-17 10:33:51.792854 /uploads/big_lots_onboarding/PYD_Signup_NoPurchase_A.csv
All_Done: 2020-09-17 10:33:51.807575
